# Handle Maven dependency issues

Explore using the task runner, to handle both maven compiler errors when dependencies are missing, and a specific task from the analyzer-lsp validator

## Setting up the environment

In [1]:
import sys

sys.modules['_elementtree'] = None
import pprint

pp = pprint.PrettyPrinter(indent=2)

## Create the Test Environment

For creating the test environment we will:

1. move the coolstore to a temp direcotry
2. Update the InventoryEntity to use UUID for the itemID as the type.

In [ ]:
import tempfile
import shutil
import os
import subprocess

#First we need to download coolstore if it does not exist.

if not os.path.exists("./coolstore"):
    subprocess.run("../../example/fetch.sh")

temp_dir = tempfile.TemporaryDirectory()
coolstore_path = os.path.join(temp_dir.name, "coolstore")
shutil.copytree("./coolstore", coolstore_path)

## make the necessary change
shutil.copyfile("./testing_pom_changes/pom.xml", f"{coolstore_path}/pom.xml")
print(temp_dir)


## Testing for Dependency issue with coolstore missing a dependencies

This will test that removed dependency is re-added 

In [ ]:

from pathlib import Path
from kai.service.llm_interfacing.model_provider import ModelProvider
from kai.models.kai_config import KaiConfig
from kai.repo_level_awareness.agent.dependency_agent.dependency_agent import MavenDependencyAgent
from kai.repo_level_awareness.api import RpcClientConfig
from kai.repo_level_awareness.codeplan import TaskManager
from kai.repo_level_awareness.task_runner.compiler.maven_validator import MavenCompileStep, SymbolNotFoundError, PackageDoesNotExistError
from kai.repo_level_awareness.task_runner.dependency.task_runner import DependencyTaskRunner
import os

from kai.repo_level_awareness.vfs.git_vfs import RepoContextManager

config = RpcClientConfig(Path(coolstore_path), None, None, None, None, None)
kai_config = KaiConfig.model_validate_filepath("../../kai/config.toml")
modelProvider = ModelProvider(kai_config.models)
rcm = RepoContextManager(config.repo_directory, modelProvider.llm)
agent = MavenDependencyAgent(modelProvider.llm, config.repo_directory)

# Create the task runner.
task_runner = DependencyTaskRunner(agent)

## Now, we need to set up code plan, with this agent, and to talk to coolstore. To show the missing dependencies we will only use the maven compiler validator.

task_manager = TaskManager(
        config,
        rcm,
        None,
        validators=[MavenCompileStep(config)],
        agents=[task_runner],
    )


for task in task_manager.get_next_task():
    pp.pprint(task)
    if isinstance(task, PackageDoesNotExistError):
        task_manager.supply_result(task_manager.execute_task(task))


## Cleanup temp dir

In [4]:
temp_dir.cleanup()